In [1]:
(require booleans redex/reduction-semantics)

In [16]:
(define/ppl q #:no-check
  ((SEL = q_SEL)
   (K0 = q_K0)
   (Kn = q_Kn)))

(SEL = q_SEL)
(K0 = q_K0)
(Kn = q_Kn)


In [17]:
(define/ppl start #:no-check
  (;; p
   (p_GO = (and GO S))
   (p_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(p_SUSP = SUSP)
   ;(p_KILL = KILL)
   ;; q
   (q_GO = (and GO (not S)))
   (q_RES = RES)
   ;; these wires are not needed for the proof, and are never read so we can 
   ;; ignore them
   ;(q_SUSP = SUSP)
   ;(q_KILL = KILL)
   ;; out
   (SEL = (or p_SEL q_SEL))
   (K0 = (or p_K0 q_K0))
   (Kn = (or p_Kn q_Kn))))
        

(p_GO = (and GO S))
(p_RES = RES)
(q_GO = (and GO (not S)))
(q_RES = RES)
(SEL = (or p_SEL q_SEL))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


By [S-consistent], we know S is false.

In [18]:
(define/ppl add-S #:no-check
  (replace* start [S false]))

(p_GO = (and GO false))
(p_RES = RES)
(q_GO = (and GO (not false)))
(q_RES = RES)
(SEL = (or p_SEL q_SEL))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


Now we do induction over instants

## Case 1: Instant 0

In [19]:
(define/ppl first-instant #:no-check
  (replace* add-S [p_SEL false]))

(p_GO = (and GO false))
(p_RES = RES)
(q_GO = (and GO (not false)))
(q_RES = RES)
(SEL = (or false q_SEL))
(K0 = (or p_K0 q_K0))
(Kn = (or p_Kn q_Kn))


Now convert to TVF

In [20]:
(define/ppl converted #:no-check
  (convert-P first-instant))

((+ p_GO) = (and (+ GO) false))
((- p_GO) = (or (- GO) true))
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = (and (+ GO) true))
((- q_GO) = (or (- GO) false))
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (or false (+ q_SEL)))
((- SEL) = (and true (- q_SEL)))
((+ K0) = (or (+ p_K0) (+ q_K0)))
((- K0) = (and (- p_K0) (- q_K0)))
((+ Kn) = (or (+ p_Kn) (+ q_Kn)))
((- Kn) = (and (- p_Kn) (- q_Kn)))


In [21]:
(define/ppl case1-simplify1
  replace* converted
  [(and (+ GO) true) (+ GO)]
  [(or (- GO) false) (- GO)]
  [(and (+ GO) false) false]
  [(or (- GO) true) true]
  [(or (+ p_SEL) false) (+ p_SEL)]
  [(and (- p_SEL) true) (- p_SEL)])

((+ p_GO) = false)
((- p_GO) = true)
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = (+ GO))
((- q_GO) = (- GO))
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (or false (+ q_SEL)))
((- SEL) = (and true (- q_SEL)))
((+ K0) = (or (+ p_K0) (+ q_K0)))
((- K0) = (and (- p_K0) (- q_K0)))
((+ Kn) = (or (+ p_Kn) (+ q_Kn)))
((- Kn) = (and (- p_Kn) (- q_Kn)))


Note that ¬( GO ∨ (RES ∧ SEL)) is true for q (as GO and SEL are false). Therefore all outputs of p are false.

In [22]:
(define/ppl all-false #:no-check
  (replace* case1-simplify1
   [(+ p_K0) false] [(- p_K0) true]
   [(+ p_Kn) false] [(- p_Kn) true]))

((+ p_GO) = false)
((- p_GO) = true)
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = (+ GO))
((- q_GO) = (- GO))
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (or false (+ q_SEL)))
((- SEL) = (and true (- q_SEL)))
((+ K0) = (or false (+ q_K0)))
((- K0) = (and true (- q_K0)))
((+ Kn) = (or false (+ q_Kn)))
((- Kn) = (and true (- q_Kn)))


In [23]:
(define/ppl case1-simplify2
  replace* all-false
  [(or false (+ q_K0)) (+ q_K0)]
  [(and true (- q_K0)) (- q_K0)]
  [(or false (+ q_Kn)) (+ q_Kn)]
  [(and true (- q_Kn)) (- q_Kn)])

((+ p_GO) = false)
((- p_GO) = true)
((+ p_RES) = (+ RES))
((- p_RES) = (- RES))
((+ q_GO) = (+ GO))
((- q_GO) = (- GO))
((+ q_RES) = (+ RES))
((- q_RES) = (- RES))
((+ SEL) = (or false (+ q_SEL)))
((- SEL) = (and true (- q_SEL)))
((+ K0) = (+ q_K0))
((- K0) = (- q_K0))
((+ Kn) = (+ q_Kn))
((- Kn) = (- q_Kn))


In [24]:
(assert-same (term case1-simplify2) (term (convert-P q)))

## Case 2: Future instants

By [sel-later] we know that in future instant's p_SEL is still false. In addition p_GO will also still be false because S is false. Thus the same reasoning applies to all instants.